# Classificação de Sentimentos com In-Context Learning (ICL)

Este notebook implementa um classificador de sentimentos utilizando **In-Context Learning** com Large Language Models (LLMs).

## O que é In-Context Learning?

In-Context Learning é uma abordagem moderna de aprendizado de máquina onde:
- **Não há treinamento**: O modelo já foi pré-treinado em bilhões de textos
- **Aprendizado via prompt**: O modelo aprende a tarefa através de exemplos no prompt
- **Zero/Few-Shot**: Pode funcionar sem exemplos (zero-shot) ou com poucos exemplos (few-shot)

## Diferenças das Abordagens Anteriores

| Aspecto | SVM/BERT (Fine-tuning) | In-Context Learning |
|---------|------------------------|---------------------|
| Treinamento | Requer treinar/fine-tunar | Não requer treinamento |
| Dados necessários | Milhares de exemplos | 0-10 exemplos no prompt |
| Tempo de setup | Horas | Minutos |
| Custo computacional | Alto (GPU) | Baixo (API call) |
| Flexibilidade | Fixo após treino | Adaptável instantaneamente |
| Custo por inferência | Baixo | Alto (API) |

## Abordagens que vamos implementar

1. **Zero-Shot**: Sem exemplos, apenas instrução
2. **One-Shot**: Um exemplo de positivo e um de negativo
3. **Few-Shot**: Múltiplos exemplos (3-5 por classe)
4. **Chain-of-Thought**: Raciocínio passo a passo

## Objetivo
Classificar avaliações do Yelp usando LLMs via API sem nenhum treinamento adicional.

## 1. Instalação e Importação de Bibliotecas

Para este notebook, vamos usar:
- **OpenAI API**: GPT-4 ou GPT-3.5-turbo (requer API key)
- **Alternativas gratuitas**: Ollama (modelos locais como Llama 3.1, Mistral)
- **Anthropic API**: Claude (opcional, também requer API key)

### Opções de API

1. **OpenAI (Pago)**: Melhor performance, requer `OPENAI_API_KEY`
2. **Ollama (Gratuito)**: Roda localmente, sem custo, mas precisa de recursos
3. **Anthropic (Pago)**: Claude, requer `ANTHROPIC_API_KEY`

Para este exemplo, vamos implementar com suporte para múltiplas APIs.

In [1]:
# Instalar bibliotecas necessárias (execute apenas uma vez)
! uv run pip install openai anthropic pandas numpy matplotlib seaborn scikit-learn tqdm

# NOTA: Para usar Ollama (gratuito, local):
# 1. Instale Ollama: https://ollama.ai
# 2. Baixe um modelo: ollama pull llama3.1
# 3. Execute: ollama serve


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm.auto import tqdm
import time
import os
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Bibliotecas básicas importadas com sucesso!")
print("\nAPIs disponíveis:")

# Tentar importar OpenAI
try:
    from openai import OpenAI
    print("✓ OpenAI API disponível")
    OPENAI_AVAILABLE = True
except ImportError:
    print("✗ OpenAI API não disponível (execute: uv run pip install openai)")
    OPENAI_AVAILABLE = False

# Tentar importar Anthropic
try:
    from anthropic import Anthropic
    print("✓ Anthropic API disponível")
    ANTHROPIC_AVAILABLE = True
except ImportError:
    print("✗ Anthropic API não disponível (execute: uv run pip install anthropic)")
    ANTHROPIC_AVAILABLE = False

# Verificar Ollama (local)
import subprocess
try:
    result = subprocess.run(['ollama', 'list'], capture_output=True, timeout=2)
    if result.returncode == 0:
        print("✓ Ollama disponível (local)")
        OLLAMA_AVAILABLE = True
    else:
        print("✗ Ollama não está rodando (execute: ollama serve)")
        OLLAMA_AVAILABLE = False
except (subprocess.TimeoutExpired, FileNotFoundError):
    print("✗ Ollama não instalado (visite: https://ollama.ai)")
    OLLAMA_AVAILABLE = False

print("\nNOTA: Este notebook funcionará com qualquer API disponível.")

✓ Anthropic API disponível
✓ Ollama disponível (local)

NOTA: Este notebook funcionará com qualquer API disponível.


In [3]:
# DEBUG: Check which Python interpreter Jupyter is using
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"\nExpected path (uv environment):")
print("/Users/jwcunha/Documents/COMPANIES/AB/repos/private/premium/phd/computer-vision-and-nlp/sentiment-analysis/.venv/bin/python")
print(f"\nMatch: {'.venv' in sys.executable}")

Python executable: /Users/jwcunha/Documents/COMPANIES/AB/repos/private/premium/phd/computer-vision-and-nlp/sentiment-analysis/.venv/bin/python
Python version: 3.13.0 (v3.13.0:60403a5409f, Oct  7 2024, 00:37:40) [Clang 15.0.0 (clang-1500.3.9.4)]

Expected path (uv environment):
/Users/jwcunha/Documents/COMPANIES/AB/repos/private/premium/phd/computer-vision-and-nlp/sentiment-analysis/.venv/bin/python

Match: True


### Soluções para Problema de Ambiente

Se o Python acima **NÃO** contém `.venv`, você tem 3 opções:

#### Opção 1: Instalar no ambiente atual do Jupyter (Rápido)
```python
!pip install openai anthropic
```

#### Opção 2: Mudar o kernel do Jupyter
1. No menu superior do Jupyter: **Kernel → Change Kernel**
2. Selecione o kernel com `.venv` no nome
3. Reinicie o kernel

#### Opção 3: Reinstalar Jupyter no ambiente correto (Recomendado)
```bash
# No terminal:
cd /Users/jwcunha/Documents/COMPANIES/AB/repos/private/premium/phd/computer-vision-and-nlp/sentiment-analysis
uv run pip install ipykernel
uv run python -m ipykernel install --user --name=sentiment-analysis
# Depois reinicie Jupyter e selecione o kernel "sentiment-analysis"
```

In [10]:
# QUICK FIX: Instalar pacotes usando uv
# Execute esta célula se precisar instalar os pacotes

import subprocess
import sys

print("Instalando pacotes usando uv...")
print(f"Ambiente: {sys.executable}\n")

packages = ['openai', 'anthropic', 'requests']

try:
    # Usar uv pip ao invés de pip normal
    for package in packages:
        result = subprocess.run(
            ['uv', 'pip', 'install', package],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            print(f"✓ {package} instalado")
        else:
            print(f"✗ Erro ao instalar {package}")
            if result.stderr:
                print(f"   {result.stderr[:200]}")
    
    print("\n✓ Instalação concluída!")
    print("⚠️  IMPORTANTE: Reinicie o kernel (Kernel → Restart Kernel) para que as mudanças tenham efeito.")
    
except FileNotFoundError:
    print("✗ Comando 'uv' não encontrado.")
    print("\nAlternativa: Execute no terminal:")
    print("  cd /Users/jwcunha/Documents/COMPANIES/AB/repos/private/premium/phd/computer-vision-and-nlp/sentiment-analysis")
    print("  uv pip install openai anthropic requests")

✓ openai instalado
✓ anthropic instalado
✓ requests instalado

✓ Instalação concluída!
⚠️  IMPORTANTE: Reinicie o kernel (Kernel → Restart Kernel) para que as mudanças tenham efeito.


In [11]:
# Configurar variáveis de ambiente (se necessário)
# Se você tiver as API keys configuradas, descomente e adicione aqui:

# import os
# os.environ['OPENAI_API_KEY'] = 'sua-chave-aqui'
# os.environ['ANTHROPIC_API_KEY'] = 'sua-chave-aqui'

# Ou carregue de um arquivo .env:
# from dotenv import load_dotenv
# load_dotenv()

print("Verificando API keys...")
import os

if os.getenv('OPENAI_API_KEY'):
    print(f"✓ OPENAI_API_KEY encontrada (primeiros 10 chars: {os.getenv('OPENAI_API_KEY')[:10]}...)")
else:
    print("✗ OPENAI_API_KEY não encontrada")

if os.getenv('ANTHROPIC_API_KEY'):
    print(f"✓ ANTHROPIC_API_KEY encontrada (primeiros 10 chars: {os.getenv('ANTHROPIC_API_KEY')[:10]}...)")
else:
    print("✗ ANTHROPIC_API_KEY não encontrada")

print("\nNOTA: Se as keys não aparecerem, você pode:")
print("1. Configurá-las no terminal: export OPENAI_API_KEY='sua-key'")
print("2. Ou configurá-las diretamente no notebook (veja código acima)")
print("3. Ou usar LM Studio/Ollama (gratuito, local) que não precisa de keys")

Verificando API keys...
✓ OPENAI_API_KEY encontrada (primeiros 10 chars: sk-proj-_g...)
✓ ANTHROPIC_API_KEY encontrada (primeiros 10 chars: sk-ant-api...)

NOTA: Se as keys não aparecerem, você pode:
1. Configurá-las no terminal: export OPENAI_API_KEY='sua-key'
2. Ou configurá-las diretamente no notebook (veja código acima)
3. Ou usar LM Studio/Ollama (gratuito, local) que não precisa de keys


## 2. Carregamento dos Dados

Carregando o mesmo dataset do Yelp usado nas outras abordagens.

In [4]:
# DEBUG: Test data loading
import pandas as pd

# Test 1: Read first few lines manually
print("Test 1: Reading raw lines")
with open('dataset/yelp_reviews.csv', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 2:
            print(f"Line {i}: {line[:100]}...")
        else:
            break

# Test 2: Try loading with pandas
print("\nTest 2: Loading with pandas")
try:
    df_test = pd.read_csv('dataset/yelp_reviews.csv', names=['label', 'text'], header=None, nrows=5)
    print(f"✓ Success! Columns: {list(df_test.columns)}")
    print(f"Shape: {df_test.shape}")
    print(f"\nFirst row label: {df_test['label'].iloc[0]}")
    print(f"First row text: {df_test['text'].iloc[0][:50]}...")
except Exception as e:
    print(f"✗ Error: {e}")

Test 1: Reading raw lines
Line 0: "2","Contrary to other reviews, I have zero complaints about the service or the prices. I have been ...
Line 1: "1","Last summer I had an appointment to get new tires and had to wait a super long time. I also wen...

Test 2: Loading with pandas
✓ Success! Columns: ['label', 'text']
Shape: (5, 2)

First row label: 2
First row text: Contrary to other reviews, I have zero complaints ...


In [6]:
# Carregando o dataset do Yelp
import pandas as pd
import numpy as np

# Ler CSV sem assumir que há cabeçalho
df = pd.read_csv(
    'dataset/yelp_reviews.csv', 
    names=['label', 'text'],
    header=None,
    encoding='utf-8'
)

print(f"✓ Dataset carregado!")
print(f"Dimensões: {df.shape}")
print(f"Colunas: {list(df.columns)}")
print(f"\nDistribuição de classes:")
print(df['label'].value_counts())

# Criar amostra balanceada (método mais simples e confiável)
SAMPLE_SIZE = 200

# Pegar exemplos de cada classe separadamente
df_neg = df[df['label'] == 1].sample(n=SAMPLE_SIZE // 2, random_state=42)
df_pos = df[df['label'] == 2].sample(n=SAMPLE_SIZE // 2, random_state=42)

# Combinar e embaralhar
df_sample = pd.concat([df_neg, df_pos], ignore_index=True)
df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n✓ Amostra criada: {len(df_sample)} exemplos")
print(f"Distribuição na amostra:")
print(df_sample['label'].value_counts())
print(f"\nPrimeiros exemplos:")
print(df_sample.head(3))

✓ Dataset carregado!
Dimensões: (38000, 2)
Colunas: ['label', 'text']

Distribuição de classes:
label
2    19000
1    19000
Name: count, dtype: int64

✓ Amostra criada: 200 exemplos
Distribuição na amostra:
label
1    100
2    100
Name: count, dtype: int64

Primeiros exemplos:
   label                                               text
0      1  I used A&R Appliance Repair for many years and...
1      1  Wow, was I disappointed by this place. It was ...
2      1  This review is for customer service attitude.\...


## 3. Configuração do Cliente LLM

Vamos configurar o cliente para usar o LLM disponível. Suportamos:
1. **LM Studio** (local, gratuito) - Detectado automaticamente em `localhost:11434`
2. **OpenAI** (pago) - Requer `OPENAI_API_KEY`
3. **Anthropic** (pago) - Requer `ANTHROPIC_API_KEY`
4. **Ollama** (local, gratuito) - Requer Ollama rodando

In [7]:
import requests

# Classe wrapper para uniformizar diferentes APIs
class LLMClient:
    def __init__(self, provider='auto', model=None, api_key=None, base_url=None):
        self.provider = provider
        self.model = model
        self.api_key = api_key
        self.base_url = base_url
        self.client = None
        
        # Auto-detectar provider disponível
        if provider == 'auto':
            self.provider = self._auto_detect_provider()
        
        self._setup_client()
    
    def _auto_detect_provider(self):
        """Detecta qual provider está disponível"""
        # Verificar LM Studio (localhost:11434)
        try:
            response = requests.get('http://localhost:11434/v1/models', timeout=2)
            if response.status_code == 200:
                print("✓ LM Studio detectado em localhost:11434")
                return 'lmstudio'
        except:
            pass
        
        # Verificar Ollama (localhost:11434)
        try:
            response = requests.get('http://localhost:11434/api/tags', timeout=2)
            if response.status_code == 200:
                print("✓ Ollama detectado")
                return 'ollama'
        except:
            pass
        
        # Verificar OpenAI API key
        if os.getenv('OPENAI_API_KEY'):
            print("✓ OpenAI API key encontrada")
            return 'openai'
        
        # Verificar Anthropic API key
        if os.getenv('ANTHROPIC_API_KEY'):
            print("✓ Anthropic API key encontrada")
            return 'anthropic'
        
        raise ValueError("Nenhum LLM provider encontrado. Configure LM Studio, Ollama, ou adicione API keys.")
    
    def _setup_client(self):
        """Configura o cliente baseado no provider"""
        if self.provider == 'lmstudio':
            from openai import OpenAI
            self.client = OpenAI(
                base_url='http://localhost:11434/v1',
                api_key='lm-studio'  # LM Studio não precisa de key real
            )
            self.model = self.model or 'local-model'
            print(f"Usando LM Studio com modelo: {self.model}")
        
        elif self.provider == 'ollama':
            from openai import OpenAI
            self.client = OpenAI(
                base_url='http://localhost:11434/v1',
                api_key='ollama'  # Ollama não precisa de key real
            )
            self.model = self.model or 'llama3.1'
            print(f"Usando Ollama com modelo: {self.model}")
        
        elif self.provider == 'openai':
            from openai import OpenAI
            self.client = OpenAI(api_key=self.api_key or os.getenv('OPENAI_API_KEY'))
            self.model = self.model or 'gpt-3.5-turbo'
            print(f"Usando OpenAI com modelo: {self.model}")
        
        elif self.provider == 'anthropic':
            from anthropic import Anthropic
            self.client = Anthropic(api_key=self.api_key or os.getenv('ANTHROPIC_API_KEY'))
            self.model = self.model or 'claude-3-haiku-20240307'
            print(f"Usando Anthropic com modelo: {self.model}")
    
    def generate(self, prompt, max_tokens=100, temperature=0.0):
        """Gera resposta do LLM"""
        if self.provider in ['lmstudio', 'ollama', 'openai']:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature
            )
            return response.choices[0].message.content.strip()
        
        elif self.provider == 'anthropic':
            response = self.client.messages.create(
                model=self.model,
                max_tokens=max_tokens,
                temperature=temperature,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.content[0].text.strip()

# Inicializar cliente
try:
    llm = LLMClient(provider='auto')
    print("\n✓ Cliente LLM configurado com sucesso!")
except Exception as e:
    print(f"\n✗ Erro ao configurar LLM: {e}")
    print("\nConfigurações sugeridas:")
    print("1. Instale LM Studio (https://lmstudio.ai) e inicie um servidor")
    print("2. Ou instale Ollama (https://ollama.ai) e execute: ollama serve")
    print("3. Ou configure: export OPENAI_API_KEY='sua-chave'")

✓ LM Studio detectado em localhost:11434
Usando LM Studio com modelo: local-model

✓ Cliente LLM configurado com sucesso!


## 4. Implementação de Estratégias de In-Context Learning

Vamos implementar e comparar diferentes estratégias de prompt:

### 4.1 Zero-Shot Learning
Sem exemplos, apenas instrução direta.

In [8]:
def create_zero_shot_prompt(review_text):
    """
    Cria um prompt zero-shot (sem exemplos)
    """
    prompt = f"""Classify the sentiment of the following restaurant review as either "positive" or "negative".
Respond with ONLY one word: "positive" or "negative".

Review: {review_text}

Sentiment:"""
    return prompt

# Testar com um exemplo
sample_review = df_sample['text'].iloc[0]
sample_label = df_sample['label'].iloc[0]

prompt = create_zero_shot_prompt(sample_review)
print("Prompt Zero-Shot:")
print("=" * 70)
print(prompt)
print("=" * 70)

# Gerar resposta
response = llm.generate(prompt, max_tokens=10, temperature=0.0)
print(f"\nResposta do LLM: {response}")
print(f"Label verdadeiro: {'positive' if sample_label == 2 else 'negative'}")


Resposta do LLM: negative
Label verdadeiro: negative


### 4.2 Few-Shot Learning
Com exemplos demonstrativos (3 positivos + 3 negativos).

In [9]:
def create_few_shot_prompt(review_text, num_examples=3):
    """
    Cria um prompt few-shot com exemplos
    """
    # Exemplos cuidadosamente selecionados (equilibrados)
    examples_positive = [
        "This place is amazing! Best food I've ever had.",
        "Excellent service and delicious meals. Highly recommend!",
        "Absolutely loved it. Will definitely come back."
    ]
    
    examples_negative = [
        "Terrible experience. Food was cold and service was rude.",
        "Worst restaurant ever. Overpriced and disgusting.",
        "Never coming back. Waited for an hour and food was awful."
    ]
    
    # Construir prompt com exemplos
    prompt = "Classify the sentiment of restaurant reviews as either \"positive\" or \"negative\".\n\n"
    
    # Adicionar exemplos
    for i in range(num_examples):
        prompt += f"Review: {examples_positive[i]}\nSentiment: positive\n\n"
        prompt += f"Review: {examples_negative[i]}\nSentiment: negative\n\n"
    
    # Adicionar review a ser classificada
    prompt += f"Review: {review_text}\nSentiment:"
    
    return prompt

# Testar few-shot
prompt_few_shot = create_few_shot_prompt(sample_review, num_examples=3)
print("Prompt Few-Shot (3 exemplos por classe):")
print("=" * 70)
print(prompt_few_shot)
print("=" * 70)

# Gerar resposta
response_few_shot = llm.generate(prompt_few_shot, max_tokens=10, temperature=0.0)
print(f"\nResposta do LLM: {response_few_shot}")
print(f"Label verdadeiro: {'positive' if sample_label == 2 else 'negative'}")


Resposta do LLM: negative
Label verdadeiro: negative


### 4.3 Chain-of-Thought (CoT) Prompting
Pede ao modelo para raciocinar passo a passo antes de classificar.

In [10]:
def create_cot_prompt(review_text):
    """
    Cria um prompt com Chain-of-Thought reasoning
    """
    prompt = f"""Classify the sentiment of the following restaurant review as either "positive" or "negative".

Think step-by-step:
1. Identify key words and phrases
2. Determine if they are positive or negative
3. Make final classification

Review: {review_text}

Analysis:"""
    return prompt

# Testar CoT
prompt_cot = create_cot_prompt(sample_review)
print("Prompt Chain-of-Thought:")
print("=" * 70)
print(prompt_cot)
print("=" * 70)

# Gerar resposta (mais tokens para permitir raciocínio)
response_cot = llm.generate(prompt_cot, max_tokens=200, temperature=0.0)
print(f"\nResposta do LLM:")
print(response_cot)
print(f"\nLabel verdadeiro: {'positive' if sample_label == 2 else 'negative'}")


Resposta do LLM:
Here's the step-by-step analysis:

**1. Identify key words and phrases**

* "satisfied" (positive)
* "break" (negative)
* "admitted" (neutral)
* "paid for new parts" (positive)
* "none of the burners worked" (negative)
* "tired of the original making things worse" (negative)
* "frustrated" (negative)
* "angry" (negative)
* "no one took responsibility" (negative)
* "totally incompetent" (negative)

**2. Determine if they are positive or negative**

* The reviewer was initially satisfied with A&R Appliance Repair, but the experience turned sour.
* The repairman broke the cooktop and caused more problems.
* The company paid for new parts to fix the issue, which is a positive step.
* However, the final result was that none of the burners worked, which is negative.
* The reviewer felt frustrated and

Label verdadeiro: negative


## 5. Avaliação Comparativa das Estratégias

Vamos avaliar cada estratégia no dataset de teste e comparar os resultados.

In [11]:
def parse_sentiment(response):
    """
    Extrai sentimento da resposta do LLM
    """
    response = response.lower().strip()
    
    # Procurar por 'positive' ou 'negative' na resposta
    if 'positive' in response:
        return 2  # Label positivo
    elif 'negative' in response:
        return 1  # Label negativo
    else:
        # Fallback: tentar identificar por palavras-chave
        positive_words = ['good', 'great', 'excellent', 'amazing', 'love']
        negative_words = ['bad', 'terrible', 'awful', 'worst', 'hate']
        
        if any(word in response for word in positive_words):
            return 2
        elif any(word in response for word in negative_words):
            return 1
        else:
            return None  # Não conseguiu classificar

def evaluate_strategy(df, prompt_fn, strategy_name, max_samples=None):
    """
    Avalia uma estratégia de prompting
    """
    if max_samples:
        df = df.head(max_samples)
    
    predictions = []
    true_labels = []
    errors = 0
    
    print(f"\nAvaliando estratégia: {strategy_name}")
    print(f"Processando {len(df)} exemplos...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            # Criar prompt
            if strategy_name == "Chain-of-Thought":
                prompt = prompt_fn(row['text'])
                response = llm.generate(prompt, max_tokens=200, temperature=0.0)
            else:
                prompt = prompt_fn(row['text'])
                response = llm.generate(prompt, max_tokens=10, temperature=0.0)
            
            # Parsear resposta
            prediction = parse_sentiment(response)
            
            if prediction is not None:
                predictions.append(prediction)
                true_labels.append(row['label'])
            else:
                errors += 1
            
            # Rate limiting (evitar sobrecarga)
            time.sleep(0.1)
        
        except Exception as e:
            print(f"Erro no exemplo {idx}: {e}")
            errors += 1
            continue
    
    # Calcular métricas
    if len(predictions) > 0:
        accuracy = accuracy_score(true_labels, predictions)
        print(f"\n{strategy_name} - Resultados:")
        print(f"  Acurácia: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"  Exemplos processados: {len(predictions)}")
        print(f"  Erros/Não classificados: {errors}")
        
        return {
            'strategy': strategy_name,
            'accuracy': accuracy,
            'predictions': predictions,
            'true_labels': true_labels,
            'errors': errors
        }
    else:
        print(f"Erro: Nenhuma predição válida para {strategy_name}")
        return None

print("Funções de avaliação criadas!")

Funções de avaliação criadas!


In [12]:
# Avaliar todas as estratégias
# NOTA: Reduzimos para 50 exemplos para economizar tempo/custo
# Aumente MAX_EVAL_SAMPLES se quiser avaliar em mais dados

MAX_EVAL_SAMPLES = 50

results = []

# 1. Zero-Shot
result_zero = evaluate_strategy(
    df_sample, 
    create_zero_shot_prompt, 
    "Zero-Shot",
    max_samples=MAX_EVAL_SAMPLES
)
if result_zero:
    results.append(result_zero)

# 2. Few-Shot (3 exemplos)
result_few = evaluate_strategy(
    df_sample,
    lambda text: create_few_shot_prompt(text, num_examples=3),
    "Few-Shot (3 exemplos)",
    max_samples=MAX_EVAL_SAMPLES
)
if result_few:
    results.append(result_few)

# 3. Chain-of-Thought
result_cot = evaluate_strategy(
    df_sample,
    create_cot_prompt,
    "Chain-of-Thought",
    max_samples=MAX_EVAL_SAMPLES
)
if result_cot:
    results.append(result_cot)

print("\n" + "="*70)
print("AVALIAÇÃO CONCLUÍDA!")
print("="*70)

## 6. Visualização e Comparação dos Resultados

In [13]:
# Comparação de acurácias
strategies = [r['strategy'] for r in results]
accuracies = [r['accuracy'] for r in results]

plt.figure(figsize=(10, 6))
bars = plt.bar(strategies, accuracies, color=['#3498db', '#2ecc71', '#e74c3c'])
plt.title('Comparação de Acurácia: Estratégias de In-Context Learning', fontsize=14, fontweight='bold')
plt.ylabel('Acurácia', fontsize=12)
plt.ylim(0, 1.0)

# Adicionar valores nas barras
for i, (bar, acc) in enumerate(zip(bars, accuracies)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{acc:.2%}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks(rotation=15, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Tabela resumo
summary_df = pd.DataFrame({
    'Estratégia': strategies,
    'Acurácia': [f"{acc:.2%}" for acc in accuracies],
    'Exemplos Processados': [r['errors'] for r in results],
    'Erros': [r['errors'] for r in results]
})

print("\nResumo dos Resultados:")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)


Resumo dos Resultados:
           Estratégia Acurácia  Exemplos Processados  Erros
            Zero-Shot   94.00%                     0      0
Few-Shot (3 exemplos)  100.00%                    13     13
     Chain-of-Thought   48.00%                     0      0


In [14]:
# Matrizes de confusão para cada estratégia
fig, axes = plt.subplots(1, len(results), figsize=(15, 4))

if len(results) == 1:
    axes = [axes]

for idx, result in enumerate(results):
    cm = confusion_matrix(result['true_labels'], result['predictions'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Negativo', 'Positivo'],
                yticklabels=['Negativo', 'Positivo'])
    axes[idx].set_title(f'{result["strategy"]}', fontweight='bold')
    axes[idx].set_ylabel('Valor Real')
    axes[idx].set_xlabel('Valor Predito')

plt.tight_layout()
plt.show()

# Relatórios de classificação detalhados
for result in results:
    print(f"\n{'='*70}")
    print(f"Relatório de Classificação: {result['strategy']}")
    print(f"{'='*70}")
    print(classification_report(
        result['true_labels'], 
        result['predictions'],
        target_names=['Negativo (1)', 'Positivo (2)']
    ))


Relatório de Classificação: Zero-Shot
              precision    recall  f1-score   support

Negativo (1)       0.90      1.00      0.95        27
Positivo (2)       1.00      0.87      0.93        23

    accuracy                           0.94        50
   macro avg       0.95      0.93      0.94        50
weighted avg       0.95      0.94      0.94        50


Relatório de Classificação: Few-Shot (3 exemplos)
              precision    recall  f1-score   support

Negativo (1)       1.00      1.00      1.00        20
Positivo (2)       1.00      1.00      1.00        17

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37


Relatório de Classificação: Chain-of-Thought
              precision    recall  f1-score   support

Negativo (1)       1.00      0.04      0.07        27
Positivo (2)       0.47      1.00      0.64        23

    accuracy                           0.48   

## 7. Comparação Final: Todas as Abordagens

Comparando In-Context Learning com as abordagens anteriores (SVM + BoW, SVM + Embeddings, BERT).

In [15]:
# Tabela comparativa completa de todas as abordagens
comparison_data = {
    'Aspecto': [
        'Representação',
        'Treinamento',
        'Dados Necessários',
        'Tempo de Setup',
        'Hardware',
        'Custo por Inferência',
        'Flexibilidade',
        'Interpretabilidade',
        'Contexto',
        'Generalização',
        'Latência',
        'Estado da Arte'
    ],
    'SVM + BoW': [
        'Vetores esparsos (5000+)',
        'Sim (minutos)',
        '10k+ exemplos',
        'Minutos',
        'CPU',
        'Muito baixo',
        'Baixa (requer retreino)',
        'Alta',
        'Nenhum',
        'Boa no domínio',
        'Baixíssima (<1ms)',
        'Baseline (2000s)'
    ],
    'SVM + Embeddings': [
        'Vetores densos (100-300)',
        'Sim (< 1 hora)',
        '10k+ exemplos',
        '< 1 hora',
        'CPU',
        'Baixo',
        'Baixa (requer retreino)',
        'Média',
        'Janela local',
        'Boa',
        'Baixa (<10ms)',
        '2013-2017'
    ],
    'BERT (Fine-tuning)': [
        'Embeddings contextuais (768)',
        'Sim (horas com GPU)',
        '1k+ exemplos',
        'Horas',
        'GPU recomendada',
        'Médio',
        'Média (requer fine-tuning)',
        'Baixa',
        'Bidirecional completo',
        'Excelente',
        'Média (50-100ms)',
        '2018-2023'
    ],
    'In-Context Learning': [
        'Processamento direto LLM',
        'Não (zero-shot possível)',
        '0-10 exemplos',
        'Minutos',
        'API (sem GPU local)',
        'Alto (API calls)',
        'Altíssima (muda prompt)',
        'Média-Alta',
        'Compreensão completa',
        'Excelente (generalista)',
        'Alta (100-500ms)',
        '2023+'
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("Comparação Completa: Todas as Abordagens")
print("="*120)
print(comparison_df.to_string(index=False))
print("="*120)

Comparação Completa: Todas as Abordagens
             Aspecto                SVM + BoW         SVM + Embeddings           BERT (Fine-tuning)      In-Context Learning
       Representação Vetores esparsos (5000+) Vetores densos (100-300) Embeddings contextuais (768) Processamento direto LLM
         Treinamento            Sim (minutos)           Sim (< 1 hora)          Sim (horas com GPU) Não (zero-shot possível)
   Dados Necessários            10k+ exemplos            10k+ exemplos                 1k+ exemplos            0-10 exemplos
      Tempo de Setup                  Minutos                 < 1 hora                        Horas                  Minutos
            Hardware                      CPU                      CPU              GPU recomendada      API (sem GPU local)
Custo por Inferência              Muito baixo                    Baixo                        Médio         Alto (API calls)
       Flexibilidade  Baixa (requer retreino)  Baixa (requer retreino)   Média (requ

## 8. Conclusões

### Resumo do In-Context Learning
- **Abordagem**: Uso de LLMs pré-treinados sem fine-tuning adicional
- **Estratégias testadas**: Zero-Shot, Few-Shot, Chain-of-Thought
- **Modelos suportados**: GPT-4, GPT-3.5, Claude, Llama (via LM Studio/Ollama)

### Vantagens do In-Context Learning

1. **Sem treinamento**: Pronto para usar instantaneamente
2. **Poucos dados**: Funciona com 0-10 exemplos
3. **Flexibilidade extrema**: Mude a tarefa apenas alterando o prompt
4. **Generalização**: LLMs entendem contexto e nuances complexas
5. **Setup rápido**: Minutos ao invés de horas/dias
6. **Sem infraestrutura**: Não precisa de GPU ou ambiente de treinamento
7. **Adaptável**: Ajuste o comportamento sem retreinar

### Desvantagens do In-Context Learning

1. **Custo por inferência**: APIs pagas podem ser caras em escala
2. **Latência**: Respostas mais lentas (100-500ms) que modelos tradicionais
3. **Dependência de API**: Requer conexão com servidor (exceto modelos locais)
4. **Menor controle**: Comportamento do modelo pode ser imprevisível
5. **Limites de contexto**: Prompts muito grandes podem exceder limites
6. **Consistência**: Pode variar levemente entre execuções

### Quando usar In-Context Learning?

**Use ICL quando:**
- Prototipagem rápida ou MVP
- Poucos dados de treinamento disponíveis (< 1000 exemplos)
- Múltiplas tarefas diferentes (flexibilidade)
- Não quer manter infraestrutura de ML
- Escala baixa-média (< 1M predições/dia)
- Contexto e nuances são importantes
- Tempo de setup é crítico

**NÃO use ICL quando:**
- Custo por inferência é crítico
- Latência precisa ser mínima (< 10ms)
- Escala muito alta (milhões de predições/dia)
- Privacidade de dados impede uso de APIs externas
- Performance precisa ser maximizada em domínio específico
- Disponibilidade 100% é essencial (APIs podem ter downtime)

### Evolução das Abordagens de NLP

1. **2000s**: BoW + ML clássico (SVM, Naive Bayes)
   - Simples, rápido, interpretável
   - Sem compreensão semântica

2. **2013-2017**: Embeddings + Deep Learning (Word2Vec, GloVe)
   - Captura semântica
   - Embeddings estáticos

3. **2018-2022**: Transformers + Fine-tuning (BERT, RoBERTa)
   - Embeddings contextuais
   - Requer treino em domínio específico

4. **2023+**: LLMs + In-Context Learning (GPT-4, Claude, Llama)
   - Zero/Few-shot learning
   - Generalização extrema
   - Sem necessidade de treino

### Recomendações Práticas

**Para Produção em Escala:**
- SVM + BoW/Embeddings (baixo custo, rápido)
- BERT fine-tuned (melhor performance)

**Para Prototipagem/MVP:**
- In-Context Learning (setup instantâneo)

**Abordagem Híbrida (Recomendado):**
1. **Fase 1**: Use ICL para validar ideia rapidamente
2. **Fase 2**: Colete dados reais e fine-tune BERT
3. **Fase 3**: Otimize com modelos menores (DistilBERT) para produção
4. **Fase 4**: Mantenha ICL para casos edge/complexos

### Próximos Passos

1. **Prompt Engineering Avançado**:
   - Self-consistency (múltiplas respostas + votação)
   - Tree-of-Thought prompting
   - Retrieval-Augmented Generation (RAG)

2. **Otimização de Custo**:
   - Cache de respostas comuns
   - Modelos menores para casos simples
   - Quantização de modelos locais

3. **Ensemble**:
   - Combinar ICL com modelos fine-tuned
   - Usar ICL para casos difíceis, SVM para fáceis

4. **Modelos Especializados**:
   - Fine-tune LLMs menores (Llama 3.1 8B)
   - Destilação de conhecimento do LLM grande

## 9. Teste Interativo com Novos Textos

Teste o modelo com seus próprios exemplos!

In [ ]:
# Função para classificar novos textos com qualquer estratégia
def classify_review(text, strategy='few-shot'):
    """
    Classifica uma avaliação usando a estratégia escolhida
    
    Args:
        text: Texto da avaliação
        strategy: 'zero-shot', 'few-shot', ou 'cot'
    """
    if strategy == 'zero-shot':
        prompt = create_zero_shot_prompt(text)
        max_tokens = 10
    elif strategy == 'few-shot':
        prompt = create_few_shot_prompt(text, num_examples=3)
        max_tokens = 10
    elif strategy == 'cot':
        prompt = create_cot_prompt(text)
        max_tokens = 200
    else:
        raise ValueError(f"Estratégia inválida: {strategy}")
    
    response = llm.generate(prompt, max_tokens=max_tokens, temperature=0.0)
    sentiment = parse_sentiment(response)
    
    return {
        'text': text,
        'strategy': strategy,
        'response': response,
        'sentiment': 'Positivo' if sentiment == 2 else 'Negativo' if sentiment == 1 else 'Indefinido',
        'sentiment_label': sentiment
    }

# Testar com exemplos personalizados
test_examples = [
    "The food was absolutely amazing and the service was impeccable!",
    "Terrible place. Never going back. Waste of money.",
    "It's okay I guess, nothing special but not terrible either.",
    "Best restaurant in town! Highly recommend the pasta!",
    "Horrible experience from start to finish. Cold food and rude staff."
]

print("Testando In-Context Learning com Novos Exemplos")
print("="*80)

for i, example in enumerate(test_examples, 1):
    print(f"\n{i}. Review: {example}")
    
    # Testar com Few-Shot (geralmente a melhor performance)
    result = classify_review(example, strategy='few-shot')
    
    print(f"   Sentimento: {result['sentiment']}")
    print(f"   Resposta do modelo: {result['response']}")
    print("-"*80)

print("\n✓ Classificação concluída!")